**Analytics questions:**
1. What factors (features) affect test scores most?


2. Are there interacting features which affect test scores?

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer

Importação do dataset e exibição do **head()**

In [ ]:
df = pd.read_csv('/kaggle/input/students-exam-scores/Expanded_data_with_more_features.csv')
df.head()

Após uma rápida olhada no dataset já conseguimos ter ideia de qual tipo de *ML* utilizaremos.
 
Quase todas as *features* (dados de entrada) que temos são categóricas com exceção da **"NrSiblings"** (número de irmãos) que são numéricas discretas.

Temos três *targets* (dados de saída), **"MathScore"**, **"ReadingScore"** e **"WritingScore"**.

Inicialmente poderíamos pensar em utilizar algum algorítimo simples de classificação, porém, temos *targets* numéricas contínuas que são notas de alunos em exames, isso nos obriga a buscar um algorítimo que seja um regressor mas que saiba trabalhar com *features* categóricas.

Nesse caso, mais a frente, escolheremos o **RandomForestRegressor**.

In [ ]:
df.describe(include='all')

Com o **describe()** podemos ver se há alguma coisa fora do padrão nos dados, aparentemente aqui não podemos ver nada fora dos padrões.

Continunando, daremos uma olhada na **info()** do *DataFrame*.

In [ ]:
df.info()

A **info()** confirma nossa percepção acima dos tipos de dados de *features* e *target*.

Podemos ver também que a quantidade de **"Non-Null"** são diferentes em algumas colunas, isso significa que temos dados **"Null"** que iremos tratar na próxima etapa durante a construção do *pipeline*.

Na próxima etapa criaremos algumas **listas** que servirão para criarmos as *pipelines* mais a frante.

Teremos **4 listas**, uma com todas as *features*, uma com todas as *targets*, uma com apenas *features* numéricas e uma com apenas *features* categóricas.

In [ ]:
# Início do pipeline de treinamento
# Criação de lista para numerical e categorical features
numerical_features = [
    'NrSiblings',
]

categorical_features = [
    'Gender',
    'EthnicGroup',
    'ParentEduc',
    'LunchType',
    'TestPrep',
    'ParentMaritalStatus',
    'PracticeSport',
    'IsFirstChild',
    'TransportMeans',
    'WklyStudyHours',
]

# Selecionando features e target

features = [
    'Gender',
    'EthnicGroup',
    'ParentEduc',
    'LunchType',
    'TestPrep',
    'ParentMaritalStatus',
    'PracticeSport',
    'IsFirstChild',
    'NrSiblings',
    'TransportMeans',
    'WklyStudyHours'
]

target = [
    'MathScore',
    'ReadingScore',
    'WritingScore'
]

Dando continuidade, podemos prosseguir para a divisão dos dados em **treino** e **teste**.

In [ ]:
# separação de dados treino e teste
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Finalmente iniciaremos a construção do nosso pipeline.

A primeira etapa é preparar o pré processamento das *features*, aqui deve ficar mais claro porque dividimos elas em categóricas e numéricas.

O motivo é que cada tipo de dado precisa de uma transformação diferente.

Para as *features* numéricas, utilizaremos *'mean'* como estratégia para alterar todos os dados **Null**.

*Detalhe: Eu gosto de utilizar a 'median' por ser menos sensível aos outliers, mas nesse caso a 'mean' teve um resultado melhor e também não temos outliers nesse conjunto numérico.*

In [ ]:
# Preprocessamento de colunas numéricas
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

Para as *features* categóricas, utilizaremos *'most_frequent'* como estratégia para alterar todos os dados **Null**.

*Detalhe: A técnica de codificação 'OneHotEncoder' é bastante utilizada quando há independência entre as 'features', no nosso caso, utilizaremos 'TargetEncoder' pois há correlação entre algumas 'features'. E também porque apresentou melhores resultados.*

In [ ]:
# Preprocessamento de colunas categóricas
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder())
])

Agora prepararemos o *pipeline* de transformação das colunas e o *pipeline* final, com o modelo de algorítimo a ser treinado.

Faremos também um *reset* no índice dos dados.

In [ ]:
# Combinando pré-processadores de colunas numéricas e categóricas
preprocessor = ColumnTransformer([
    ('numeric', numeric_transformer, numerical_features),
    ('categorical', categorical_transformer, categorical_features)
])

# Criando o pipeline com etapas de pré-processamento e modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=0))
])

# Resetando o índice
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

E por fim, **a última etapa**.

Nessa fase, criaremos um laço de repetição para utilizarmos o *pipeline* de acordo com a quantidade de *targets* que temos, e no final, avaliaremos o desempenho do nosso modelo.

In [ ]:
# Treinando o pipeline para cada target
for t in target:
    pipeline.fit(X_train, y_train[t])
    y_pred = pipeline.predict(X_test)

    # Avaliando o desempenho do modelo
    r2 = r2_score(y_test[t], y_pred)
    mae = mean_absolute_error(y_test[t], y_pred)
    mse = mean_squared_error(y_test[t], y_pred)

    print(f'Target: {t}')
    print(f'R2: {r2:.2f}')
    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print('--------------------')